# 📝 Lab #3: 서울시 따릉이 대여량 예측 앙상블 모델 구축

이번 실습 과제에서는 **서울시 공공자전거(따릉이) 대여량 예측** 데이터셋을 사용하여, 오늘 배운 **스태킹(Stacking)** 기법을 **처음부터 직접 구현**해보겠습니다. `scikit-learn`의 `StackingRegressor`를 사용하지 않고, K-Fold 교차 검증을 이용한 Out-of-Fold(OOF) 예측 생성 과정을 직접 코딩하여 스태킹의 내부 동작 원리를 완벽하게 이해하는 것이 목표입니다.

### 과제 목표
1. 서울시 따릉이 데이터를 불러오고 기본적인 전처리를 수행합니다.
2. K-Fold 교차 검증 로직을 직접 구현하여 기본 모델들의 **Out-of-Fold(OOF) 예측값**을 생성합니다. (이것이 메타 모델의 훈련 데이터가 됩니다.)
3. 테스트 데이터에 대한 예측값은 각 Fold에서 생성된 모델들의 예측 평균으로 만듭니다.
4. OOF 예측값을 특성으로 사용하여 메타 모델을 학습시킵니다.
5. 학습된 메타 모델을 사용하여 3번에서 구한 테스트 데이터 예측값을 변환, 최종 예측을 만듭니다.
6. 최종 예측 결과의 **RMSLE(Root Mean Squared Log Error)**를 계산하고 성능을 평가합니다.

### Step 0: 라이브러리 임포트 및 데이터 로드

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, train_test_split
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_log_error

# RMSLE 계산을 위한 함수
def rmsle(y_true, y_pred):
    return np.sqrt(mean_squared_log_error(y_true, y_pred))

# 데이터 로드
path = '../../datasets/ml/bike-sharing/SeoulBikeData.csv'
df = pd.read_csv(path, encoding='cp949')

# 데이터 전처리
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['Hour'] = df['Hour'] # Hour는 이미 있음

df.drop(['Date', 'Seasons', 'Holiday', 'Functioning Day'], axis=1, inplace=True)

X = df.drop('Rented Bike Count', axis=1)
y = df['Rented Bike Count']

# 음수 값을 0으로 변환 (로그 변환 시 오류 방지)
y[y < 0] = 0

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("데이터 준비 완료!")
X_train.head()

### Step 1: 기본 모델(Level-0) 및 메타 모델(Level-1) 정의

스태킹에 사용할 기본 모델들과 이들의 예측을 조합할 메타 모델을 정의합니다.

In [ ]:
# TODO: 기본 모델 3개와 메타 모델 1개를 정의하세요.
# 기본 모델: RandomForest, XGBoost, LightGBM (모두 random_state=42로 고정)
# 메타 모델: LinearRegression
rf_model = ...
xgb_model = ...
lgbm_model = ...

meta_model = ...

base_models = [rf_model, xgb_model, lgbm_model]

### Step 2: K-Fold를 이용한 Out-of-Fold(OOF) 예측 생성

이제 스태킹의 핵심인 OOF 예측값을 생성합니다. `KFold`를 사용하여 훈련 데이터를 나누고, 각 Fold에서 모델을 학습시킨 후 보지 않은 데이터(validation fold)에 대한 예측을 수행합니다.

In [ ]:
# K-Fold 설정
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# OOF 예측값을 저장할 배열 초기화 (훈련 데이터 크기 x 모델 개수)
# 테스트 데이터 예측값을 저장할 배열 초기화 (테스트 데이터 크기 x 모델 개수)
oof_preds = np.zeros((X_train.shape[0], len(base_models)))
test_preds = np.zeros((X_test.shape[0], len(base_models)))

print("OOF 예측 생성을 시작합니다...")
# 각 기본 모델에 대해 K-Fold 학습 및 예측 수행
for i, model in enumerate(base_models):
    print(f"모델 {model.__class__.__name__} 학습 중...")
    
    # K-Fold 반복
    for fold, (train_idx, val_idx) in enumerate(kf.split(X_train, y_train)):
        # Fold 데이터 분리
        X_train_fold, X_val_fold = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_train_fold, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]
        
        # TODO: 모델 학습
        # model을 X_train_fold와 y_train_fold로 학습시키세요.
        # hint: model.fit( ... )
        ...
        
        # TODO: OOF 예측값 생성
        # 학습된 모델로 X_val_fold를 예측하여 oof_preds의 해당 위치(val_idx, i)에 저장하세요.
        # hint: oof_preds[val_idx, i] = model.predict( ... )
        ...

        # TODO: 테스트 데이터 예측값 생성 (각 Fold 모델의 예측을 누적)
        # 학습된 모델로 X_test를 예측하고, 그 결과를 test_preds의 i번째 열에 더해줍니다. (나중에 평균을 낼 예정)
        # hint: test_preds[:, i] += model.predict( ... )
        ...
        
        print(f"  - Fold {fold+1} 완료")

# TODO: 테스트 데이터 예측값의 평균 계산
# test_preds는 각 Fold에서 예측한 값들이 더해져 있으므로, Fold의 개수(kf.n_splits)로 나누어 평균을 구합니다.
test_preds /= ...

print("\nOOF 예측 생성 완료!")
print("OOF 예측 데이터 shape:", oof_preds.shape)
print("Test 예측 데이터 shape:", test_preds.shape)

### Step 3: 메타 모델 학습 및 최종 예측

이제 `oof_preds`를 훈련 데이터로, `y_train`을 타겟으로 하여 메타 모델을 학습시킵니다. 
그 후, 학습된 메타 모델을 사용하여 기본 모델들이 테스트 데이터에 대해 예측한 `test_preds`를 변환하여 최종 결과를 얻습니다.

In [ ]:
# TODO: 메타 모델을 oof_preds와 y_train으로 학습시키세요.
meta_model.fit(..., ...)

# TODO: 학습된 메타 모델을 사용하여 test_preds를 변환, 최종 예측값을 생성하세요.
final_predictions = meta_model.predict(...)

# 예측 결과에 음수 값이 포함될 수 있으므로 0으로 처리
final_predictions[final_predictions < 0] = 0

# RMSLE로 최종 성능 평가
final_rmsle = rmsle(y_test, final_predictions)

print(f"\n수동 구현 스태킹 모델의 최종 RMSLE: {final_rmsle:.4f}")

### Step 4: 결과 분석 및 시각화

마지막으로 실제값과 예측값을 산점도로 시각화하여 모델의 성능을 직관적으로 확인해봅시다.

In [ ]:
# TODO: Plotly를 사용하여 실제값(y_test)과 예측값(final_predictions)의 산점도를 그리세요.
# x축은 실제값, y축은 예측값으로 설정합니다.
# 완벽한 예측을 나타내는 y=x 직선(빨간색)을 추가하여 비교해보세요.
fig = px.scatter(
    x=..., 
    y=..., 
    labels={'x': '실제 대여량', 'y': '예측 대여량'},
    title='스태킹 모델 예측 결과: 실제값 vs. 예측값',
    opacity=0.5
)

fig.add_shape(
    type='line',
    x0=y_test.min(), y0=y_test.min(),
    x1=y_test.max(), y1=y_test.max(),
    line=dict(color='Red', width=2, dash='dash')
)

fig.update_layout(width=800, height=600)
fig.show()

# [결과 분석]
# 위 그래프에서 포인트들이 빨간색 점선에 가까울수록 모델의 예측이 정확하다는 의미입니다.
# RMSLE 점수와 그래프를 종합하여 스태킹 모델의 성능에 대해 자신의 생각을 정리해보세요.
# (예: 전반적으로 대여량이 많은 구간보다 적은 구간에서 예측을 더 잘하는 경향이 있는가? 등)